In [10]:
##Cheking the response
import requests
from bs4 import BeautifulSoup
url = 'https://www.rottentomatoes.com/'
response = requests.get(url)
print(response)
soup = BeautifulSoup(response.content, 'html.parser')
#print('soup')


<Response [200]>


In [34]:
##Printing the html raw data/view source for rottentomatoes

#data = response.json()
print(response.text)

<!DOCTYPE html>
<html lang="en" dir="ltr" xmlns="http://www.w3.org/1999/xhtml" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/">
    <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
        
        
            <script
                charset="UTF-8"
                crossorigin="anonymous"
                data-domain-script="7e979733-6841-4fce-9182-515fac69187f"
                integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN"
                src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js"
                type="text/javascript"
            >
            </script>
            <script type="text/javascript">
                function OptanonWrapper() { }
            </script>
        

        
            <script
                ccpa-opt-out-ids="USP"
                ccpa-opt-out-geo="US"
                ccpa-opt-out-lspa="fals

In [30]:
## ETL Pipeline (1)

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

OMDB_API_KEY = "YOUR_OMDB_API_KEY"
HEADERS = {"User-Agent": "Mozilla/5.0"}


def fetch_movies(url, selector, attr=None):
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        return [i.get(attr, "").split("/")[-1].replace("-", " ").title() if attr else i.text.strip() for i in soup.select(selector)][:10]
    return []


def fetch_movie_details(title):
    url = f"http://www.omdbapi.com/?t={title}&apikey={OMDB_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200 and response.json().get("Response") == "True":
        data = response.json()
        return {
            "Title": data.get("Title", "N/A"),
            "Year": data.get("Year", "N/A"),
            "Genre": data.get("Genre", "N/A"),
            "IMDB Rating": data.get("imdbRating", "N/A"),
            "Box Office": data.get("BoxOffice", "N/A"),
            "Release Date": pd.to_datetime(data.get("Released", "N/A"), errors='coerce')
        }
    return None


def etl_pipeline():
    movies = set(fetch_movies("https://www.boxofficemojo.com/weekend/", "td.a-text-left.mojo-field-type-release a") +
                 fetch_movies("https://www.rottentomatoes.com/browse/movies_in_theaters", "a.js-tile-link", "href"))
    df = pd.DataFrame([fetch_movie_details(movie) for movie in movies if movie])
    df.to_csv("movies_data.csv", index=False)
    print(df.head())


if __name__ == "__main__":
    etl_pipeline()

      0
0  None
1  None
2  None
3  None
4  None


In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import quote

OMDB_API_KEY = "594ea12e"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def clean_title(title):
    title = re.sub(r"[^a-zA-Z0-9 ]", "", title)  # Remove special characters
    return title.title().strip()  # Convert to title case and trim spaces


def transform_release_date(date):
    if date and date != "N/A":
        return pd.to_datetime(date, errors='coerce').strftime('%Y-%m-%d')
    return "Unknown"


def transform_genre(genre):
    if genre and genre != "N/A":
        return list(set([g.strip().lower() for g in genre.split(",")]))
    return []


def transform_imdb_rating(rating):
    try:
        return round(float(rating), 1) if rating and rating != "N/A" else None
    except ValueError:
        return None


def transform_actors(actors):
    return ", ".join(sorted(actors.split(", ")[:3])) if actors and actors != "N/A" else "Unknown"


def transform_box_office(box_office):
    return int(re.sub(r"[^0-9]", "", box_office)) if box_office and box_office != "N/A" else 0


def transform_awards(awards):
    return sum(map(int, re.findall(r'\d+', awards))) if awards and awards != "N/A" else 0


def transform_metascore(metascore):
    return round(int(metascore) / 100, 2) if metascore and metascore.isdigit() else None


def transform_language(language):
    return language.lower() if language and language != "N/A" else "Unknown"


def transform_production(production):
    return re.sub(r"[^a-zA-Z0-9 ]", "", production).strip() if production and production != "N/A" else "Independent"


def fetch_movies(url, selector, attr=None):
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        movies = [i.get(attr, "").split("/")[-1].replace("-", " ").title() if attr else i.text.strip() for i in soup.select(selector)][:10]
        print(f"Fetched movies from {url}: {movies}")  # Debugging output
        return movies
    print(f"Failed to fetch movies from {url}")
    return []


def fetch_movie_details(title):
    encoded_title = quote(title)
    url = f"http://www.omdbapi.com/?t={encoded_title}&apikey={OMDB_API_KEY}"
    response = requests.get(url)
    data = response.json()
    print(f"API Response for {title}: {data}")  # Debugging output
    
    if response.status_code == 200 and data.get("Response") == "True":
        return {
            "Title": data.get("Title", "N/A"),
            "Year": data.get("Year", "N/A"),
            "Genre": data.get("Genre", "N/A"),
            "IMDB Rating": data.get("imdbRating", "N/A"),
            "Box Office": data.get("BoxOffice", "Unknown"),
            "Release Date": pd.to_datetime(data.get("Released", "N/A"), errors='coerce')
        }
    print(f"Failed to fetch details for: {title}")
    return None


def etl_pipeline():
    movies = set(fetch_movies("https://www.boxofficemojo.com/weekend/", "td.a-text-left.mojo-field-type-release a") +
                 fetch_movies("https://www.rottentomatoes.com/browse/movies_in_theaters", "a.js-tile-link", "href"))
    
    movie_data = [fetch_movie_details(movie) for movie in movies if movie]
    df = pd.DataFrame([m for m in movie_data if m is not None])
    df.to_csv("movies_data.csv", index=False)
    print(df.head())


if __name__ == "__main__":
    etl_pipeline()


Fetched movies from https://www.boxofficemojo.com/weekend/: ['Novocaine', 'Mickey 17', 'Captain America: Brave New World', 'Captain America: Brave New World', 'Captain America: Brave New World', 'Captain America: Brave New World', 'Dog Man', 'Dog Man', 'Flight Risk', 'Mufasa: The Lion King']
Fetched movies from https://www.rottentomatoes.com/browse/movies_in_theaters: ['Zerobaseone_The_First_Tour_Timeless_World', 'High_Rollers', 'Young_Hearts_2024', 'Raging_Midlife_2025', 'The_Diplomat', 'El', 'Court_State_Vs_A_Nobody', 'Papa_2024', 'Six_Each', 'Goodbye_Horses_The_Many_Lives_Of_Q_Lazzarus']
API Response for Dog Man: {'Title': 'Dog Man', 'Year': '2025', 'Rated': 'PG', 'Released': '31 Jan 2025', 'Runtime': '89 min', 'Genre': 'Animation, Action, Adventure', 'Director': 'Peter Hastings', 'Writer': 'Peter Hastings, Dav Pilkey', 'Actors': 'Pete Davidson, Poppy Liu, Lil Rel Howery', 'Plot': 'Dog Man, half dog and half man, he is sworn to protect and serve as he doggedly pursues the feline sup

In [52]:
pip install fpdf

In [54]:
##ETL Pipeline(2)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from urllib.parse import quote

# Ensure fpdf is installed
try:
    from fpdf import FPDF
except ModuleNotFoundError:
    import os
    os.system('pip install fpdf')
    from fpdf import FPDF

OMDB_API_KEY = "YOUR_OMDB_API_KEY"
HEADERS = {"User-Agent": "Mozilla/5.0"}


def clean_title(title):
    title = re.sub(r"[^a-zA-Z0-9 ]", "", title)  # Remove special characters
    return title.title().strip()  # Convert to title case and trim spaces


def transform_release_date(date):
    if date and date != "N/A":
        return pd.to_datetime(date, errors='coerce').strftime('%Y-%m-%d')
    return "Unknown"


def transform_genre(genre):
    if genre and genre != "N/A":
        return list(set([g.strip().lower() for g in genre.split(",")]))
    return []


def transform_imdb_rating(rating):
    try:
        return round(float(rating), 1) if rating and rating != "N/A" else None
    except ValueError:
        return None


def transform_actors(actors):
    return ", ".join(sorted(actors.split(", ")[:3])) if actors and actors != "N/A" else "Unknown"


def transform_box_office(box_office):
    return int(re.sub(r"[^0-9]", "", box_office)) if box_office and box_office != "N/A" else 0


def transform_awards(awards):
    return sum(map(int, re.findall(r'\d+', awards))) if awards and awards != "N/A" else 0


def transform_metascore(metascore):
    return round(int(metascore) / 100, 2) if metascore and metascore.isdigit() else None


def transform_language(language):
    return language.lower() if language and language != "N/A" else "Unknown"


def transform_production(production):
    return re.sub(r"[^a-zA-Z0-9 ]", "", production).strip() if production and production != "N/A" else "Independent"


def fetch_movies(url, selector, attr=None):
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        movies = [clean_title(i.get(attr, "").split("/")[-1].replace("-", " ").title() if attr else i.text.strip()) for i in soup.select(selector)][:10]
        print(f"Fetched movies from {url}: {movies}")
        return movies
    print(f"Failed to fetch movies from {url}")
    return []


def fetch_movie_details(title):
    encoded_title = quote(title)
    url = f"http://www.omdbapi.com/?t={encoded_title}&apikey={OMDB_API_KEY}"
    response = requests.get(url)
    data = response.json()
    print(f"API Response for {title}: {data}")
    
    if response.status_code == 200 and data.get("Response") == "True":
        return {
            "Title": clean_title(data.get("Title", "N/A")),
            "Year": data.get("Year", "N/A"),
            "Genre": transform_genre(data.get("Genre", "N/A")),
            "IMDB Rating": transform_imdb_rating(data.get("imdbRating", "N/A")),
            "Box Office": transform_box_office(data.get("BoxOffice", "N/A")),
            "Release Date": transform_release_date(data.get("Released", "N/A")),
            "Actors": transform_actors(data.get("Actors", "N/A")),
            "Awards": transform_awards(data.get("Awards", "N/A")),
            "Metascore": transform_metascore(data.get("Metascore", "N/A")),
            "Language": transform_language(data.get("Language", "N/A")),
            "Production": transform_production(data.get("Production", "N/A")),
        }
    print(f"Failed to fetch details for: {title}")
    return None


def generate_pdf(df):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(200, 10, "Movie Data Report", ln=True, align="C")
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    
    for _, row in df.iterrows():
        for col in df.columns:
            pdf.cell(40, 10, f"{col}: {row[col]}", ln=True)
        pdf.ln(5)
    
    pdf.output("movies_report.pdf")


def etl_pipeline():
    print("Extracting movies from Box Office Mojo...")
    movies = fetch_movies("https://www.boxofficemojo.com/weekend/", "td.a-text-left.mojo-field-type-release a")
    
    print("Extracting movies from Rotten Tomatoes...")
    movies += fetch_movies("https://www.rottentomatoes.com/browse/movies_in_theaters", "a.js-tile-link", "href")
    
    print("Fetching movie details from OMDb API...")
    movie_data = [fetch_movie_details(movie) for movie in set(movies) if movie]
    
    print("Transforming and saving data...")
    df = pd.DataFrame([m for m in movie_data if m is not None])
    df.to_csv("movies_data.csv", index=False)
    generate_pdf(df)
    print("ETL pipeline completed. Data saved to CSV and PDF.")


if __name__ == "__main__":
    etl_pipeline()


Extracting movies from Box Office Mojo...
Fetched movies from https://www.boxofficemojo.com/weekend/: ['Novocaine', 'Mickey 17', 'Captain America Brave New World', 'Captain America Brave New World', 'Captain America Brave New World', 'Captain America Brave New World', 'Dog Man', 'Dog Man', 'Flight Risk', 'Mufasa The Lion King']
Extracting movies from Rotten Tomatoes...
Fetched movies from https://www.rottentomatoes.com/browse/movies_in_theaters: ['Centeredjoelieberman', 'Aumthecultattheendoftheworld', 'Hoodwitch', 'Layla']
Fetching movie details from OMDb API...
API Response for Dog Man: {'Response': 'False', 'Error': 'Invalid API key!'}
Failed to fetch details for: Dog Man
API Response for Captain America Brave New World: {'Response': 'False', 'Error': 'Invalid API key!'}
Failed to fetch details for: Captain America Brave New World
API Response for Novocaine: {'Response': 'False', 'Error': 'Invalid API key!'}
Failed to fetch details for: Novocaine
API Response for Mufasa The Lion King

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from urllib.parse import quote

# Ensure fpdf is installed
try:
    from fpdf import FPDF
except ModuleNotFoundError:
    import os
    os.system('pip install fpdf')
    from fpdf import FPDF

OMDB_API_KEY = "594ea12e"
HEADERS = {"User-Agent": "Mozilla/5.0"}


def clean_title(title):
    title = re.sub(r"[^a-zA-Z0-9 ]", "", title)  # Remove special characters
    return title.title().strip()  # Convert to title case and trim spaces


def transform_release_date(date):
    if date and date != "N/A":
        return pd.to_datetime(date, errors='coerce').strftime('%Y-%m-%d')
    return "Unknown"


def transform_genre(genre):
    if genre and genre != "N/A":
        return list(set([g.strip().lower() for g in genre.split(",")]))
    return []


def transform_imdb_rating(rating):
    try:
        return round(float(rating), 1) if rating and rating != "N/A" else None
    except ValueError:
        return None


def transform_actors(actors):
    return ", ".join(sorted(actors.split(", ")[:3])) if actors and actors != "N/A" else "Unknown"


def transform_box_office(box_office):
    return int(re.sub(r"[^0-9]", "", box_office)) if box_office and box_office != "N/A" else 0


def transform_awards(awards):
    return sum(map(int, re.findall(r'\d+', awards))) if awards and awards != "N/A" else 0


def transform_metascore(metascore):
    return round(int(metascore) / 100, 2) if metascore and metascore.isdigit() else None


def transform_language(language):
    return language.lower() if language and language != "N/A" else "Unknown"


def transform_production(production):
    return re.sub(r"[^a-zA-Z0-9 ]", "", production).strip() if production and production != "N/A" else "Independent"


def fetch_movies(url, selector, attr=None):
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        movies = [clean_title(i.get(attr, "").split("/")[-1].replace("-", " ").title() if attr else i.text.strip()) for i in soup.select(selector)][:10]
        print(f"Fetched movies from {url}: {movies}")
        return movies
    print(f"Failed to fetch movies from {url}")
    return []


def fetch_movie_details(title):
    encoded_title = quote(title)
    url = f"http://www.omdbapi.com/?t={encoded_title}&apikey={OMDB_API_KEY}"
    response = requests.get(url)
    data = response.json()
    print(f"API Response for {title}: {data}")
    
    if response.status_code == 200 and data.get("Response") == "True":
        return {
            "Title": clean_title(data.get("Title", "N/A")),
            "Year": data.get("Year", "N/A"),
            "Genre": transform_genre(data.get("Genre", "N/A")),
            "IMDB Rating": transform_imdb_rating(data.get("imdbRating", "N/A")),
            "Box Office": transform_box_office(data.get("BoxOffice", "N/A")),
            "Release Date": transform_release_date(data.get("Released", "N/A")),
            "Actors": transform_actors(data.get("Actors", "N/A")),
            "Awards": transform_awards(data.get("Awards", "N/A")),
            "Metascore": transform_metascore(data.get("Metascore", "N/A")),
            "Language": transform_language(data.get("Language", "N/A")),
            "Production": transform_production(data.get("Production", "N/A")),
        }
    print(f"Failed to fetch details for: {title}")
    return None


def generate_pdf(df):
    if df.empty:
        print("⚠️ No data available for PDF report.")
        return
    
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(200, 10, "Movie Data Report", ln=True, align="C")
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    
    for _, row in df.iterrows():
        for col in df.columns:
            pdf.cell(40, 10, f"{col}: {row[col] if pd.notna(row[col]) else 'N/A'}", ln=True)
        pdf.ln(5)
    
    pdf.output("movies_report.pdf")
    print("✅ PDF file saved successfully.")


def etl_pipeline():
    print("Extracting movies from Box Office Mojo...")
    movies = fetch_movies("https://www.boxofficemojo.com/weekend/", "td.a-text-left.mojo-field-type-release a")
    
    print("Extracting movies from Rotten Tomatoes...")
    movies += fetch_movies("https://www.rottentomatoes.com/browse/movies_in_theaters", "a.js-tile-link", "href")
    
    print("Fetching movie details from OMDb API...")
    movie_data = [fetch_movie_details(movie) for movie in set(movies) if movie]
    
    if not movie_data:
        print("⚠️ No movie data found. Exiting ETL pipeline.")
        return
    
    print("Transforming and saving data...")
    df = pd.DataFrame([m for m in movie_data if m is not None])
    
    if df.empty:
        print("⚠️ DataFrame is empty. Nothing to save.")
        return
    
    print(f"✅ Saving {len(df)} movies to CSV and PDF...")
    
    try:
        df.to_csv("movies_data.csv", index=False, encoding="utf-8")
        print("✅ CSV file saved successfully.")
    except Exception as e:
        print(f"❌ Failed to save CSV: {e}")
    
    generate_pdf(df)
    print("🎬 ETL pipeline completed.")


if __name__ == "__main__":
    etl_pipeline()


Extracting movies from Box Office Mojo...
Fetched movies from https://www.boxofficemojo.com/weekend/: ['Novocaine', 'Mickey 17', 'Captain America Brave New World', 'Captain America Brave New World', 'Captain America Brave New World', 'Captain America Brave New World', 'Dog Man', 'Dog Man', 'Flight Risk', 'Mufasa The Lion King']
Extracting movies from Rotten Tomatoes...
Fetched movies from https://www.rottentomatoes.com/browse/movies_in_theaters: ['Centeredjoelieberman', 'Aumthecultattheendoftheworld', 'Hoodwitch', 'Layla']
Fetching movie details from OMDb API...
API Response for Dog Man: {'Title': 'Dog Man', 'Year': '2025', 'Rated': 'PG', 'Released': '31 Jan 2025', 'Runtime': '89 min', 'Genre': 'Animation, Action, Adventure', 'Director': 'Peter Hastings', 'Writer': 'Peter Hastings, Dav Pilkey', 'Actors': 'Pete Davidson, Poppy Liu, Lil Rel Howery', 'Plot': 'Dog Man, half dog and half man, he is sworn to protect and serve as he doggedly pursues the feline supervillain Petey the Cat.', 'L

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()